In [ ]:
from poitousprint import get_online_csv, Toflit
from vega import VegaLite
import pandas as pds

toflit_client = Toflit()

reexports = get_online_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQ1X06hRIbgExe7SOxAApZAHnfW6hGD94v0BPIBLHm4QvCbdBT8mqoMtml0ZrSkQlUbU1LqKuU5T2JF/pub?output=csv')

reexports_map = {}
for r in reexports:
    product = r['revolutionempire']
    retype = r['reexportations']
    reexports_map[product] = retype

In [ ]:
# nb sur les sources :
# best_guess_national_region -> valeur du commerce total de chaque région
# best_guess_region_prodxpart -> par produit et partenaire

all_flows = toflit_client.get_flows(best_guess_region_prodxpart='1')

In [ ]:
def normalize_etype(f):
    itype = f['export_import']
    if itype == 'Imports' or itype == 'import':
        itype = 'import'
    elif itype == 'Exports':
        itype = 'export'
    f['export_import'] = itype
    return f
all_flows = [normalize_etype(f) for f in all_flows]
values_per_year_x_direction = {}

for f in all_flows:
    direction = f['customs_region']
    value = f['value'] if f['value'] != '' else 0
    # value = int(value.split('.')[0])
    itype = f['export_import']
    year = f['year']
    if year not in values_per_year_x_direction:
        values_per_year_x_direction[year] = {}
    if direction not in values_per_year_x_direction[year]:
        values_per_year_x_direction[year][direction] = {
            "import": 0,
            "export": 0,
            "total": 0
        }
    values_per_year_x_direction[year][direction]["total"] += float(value)
    values_per_year_x_direction[year][direction][itype] += float(value)


In [ ]:


def process_reexports(flow):
    p = flow['product_revolutionempire']
    if p in reexports_map:
        return reexports_map[p]
    else:
        return 'Non renseigné'
    
def prepare_reexports(flows, itype="total"):
    directions = {}
    for flow in flows :
        direction = flow['customs_region']
        etype = process_reexports(flow)
        year = flow['year']
        value = flow['value'] if flow['value'] != '' else 0
        if direction not in directions:
            directions[direction] = {}
        if etype not in directions[direction]:
            directions[direction][etype] = 0
        directions[direction][etype] += float(value)
    output = []
    directions = [{"direction": d, "values": v} for d, v in directions.items()]
    for d in directions:
        for etype, value in d['values'].items():
            output.append({
                "direction": d['direction'], 
                "etype": etype, 
                "value_total": value,
                "value_part": value / values_per_year_x_direction[year][d['direction']][itype],
                "value_year_direction": values_per_year_x_direction[year][d['direction']][itype],
                "year": year
            })
    return output

VegaLite({
    "title": "Part des réexports par direction des fermes en 1750",
    "mark": "rect",
    "encoding": {
        "x": {
            "type": "nominal", 
            "field": "etype",
            "title": "réexportation"
        },
        "y": {
            "field": "direction",
            "type": "nominal",
            "title": "direction des fermes",
            "sort": "-x"
        },
        "color": {
            "type": "quantitative",
            "field": "value_part"
        }
    }
}, pds.DataFrame(prepare_reexports([f for f in all_flows if f["year"] == '1750'])))



In [ ]:
VegaLite({
    "title": "Part des réexports par direction des fermes en 1789",
    "mark": "rect",
    "encoding": {
        "x": {
            "type": "nominal", 
            "field": "etype",
            "title": "réexportation"
        },
        "y": {
            "field": "direction",
            "type": "nominal",
            "title": "direction des fermes",
            "sort": "-x"
        },
        "color": {
            "type": "quantitative",
            "field": "value_part"
        }
    }
}, pds.DataFrame(prepare_reexports([f for f in all_flows if f["year"] == '1789'])))

In [ ]:
def prepare_reexports_by_year(flows, itype="total"):
    directions = {}
    for flow in flows :
        direction = flow['customs_region']
        etype = process_reexports(flow)
        year = flow['year']
        value = flow['value'] if flow['value'] != '' else 0
        if direction not in directions:
            directions[direction] = {}
        if year not in directions[direction]:
            directions[direction][year] = {}
        if etype not in directions[direction][year]:
            directions[direction][year][etype] = 0
        directions[direction][year][etype] += float(value)
    output = []
    for direction, years in directions.items():
        for year, etypes in years.items():
            for etype, value in etypes.items():
                output.append({
                    "direction": direction, 
                    "etype": etype, 
                    "value_total": value,
                    "value_part": value / values_per_year_x_direction[year][direction][itype],
                    "value_year_direction": values_per_year_x_direction[year][direction][itype],
                    "year": year
                })
    return output

VegaLite({
    "title": "Évolution des parts de réexportation par direction et par année",
    "mark": "line",
    "encoding": {
        "x": {
            "type": "ordinal", 
            "field": "year",
            "title": "année"
        },
        "y": {
            "field": "value_part",
            "type": "quantitative",
            "title": "portion de réexport dans les échanges"
        },
        "color": {
            "type": "nominal",
            "field": "direction",
            "title": "Direction des fermes"
        }
    }
}, pds.DataFrame([f for f in prepare_reexports_by_year(all_flows) if f["etype"] == "Réexportation"]))